In [84]:
from elasticsearch import Elasticsearch
import numpy as np
from datetime import datetime
es = Elasticsearch('http://localhost:9200')
import json
import time
import copy
import csv
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

# try:
#     print('try to create an index')
#     es.indices.create(index='question_pool_all', ignore=400)
#     print('Create the index succesffuly.')
# except:
#     print('error')
# # es.indices.delete(index='question_pool_all', ignore=[400, 404])
# print('Recreating the index.')
# es.indices.create(index='question_pool_all', ignore=400)
# print('Do not rebuild the dataset, it takes hours.')

start_time = time.time()
count = 1
# './question_base/pool2(feedback_request_post_comments).json'
# './question_base/question_pool2/pool2.json'
with open('./question_base/pool1_0824.json', 'rb') as f:
    items = json.load(f)
    items_length = len(items)
    


[nltk_data] Downloading package punkt to /Users/zhenhui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
import pandas as pd

# convert into dataframe
df = pd.read_excel("./quiz1_process.xlsx")
dataset = []
# convert into dictionary
dataset = df.to_dict('records')

In [93]:
store_items = []
count = 0
for item in items:
    for question in item['questions']:
        flag = False
        good_question = {}
        for i in range(len(dataset)):
            if question['question'] == dataset[i]['Question']:
                flag = True
                count += 1
                good_question = dataset[i]
        if flag:
            store_item = copy.deepcopy(item)
            # print(question)
            store_item['questions'] = ''
            store_item['question_id'] = question['question_id:']
            store_item['raw_critique'] = question['raw_critique']
            store_item['select_design_concept'] = question['select_design_concept']
            store_item['mention_ui_elements'] = question['mention_ui_elements']
            store_item['question'] = question['question']
            store_item['right_answer'] = good_question['Correct Answer']
            store_item['explanation'] = question['explanation']
            # store_item['other_options'] = question['other_options']
            # store_item['other_options_wiki'] = question['other_options_wiki']

            store_item['ui_elements'] = ''
            for ui_element in question['mention_ui_elements']:
                store_item['ui_elements'] += ' ' + ui_element['name']

            # new structure 20220820
            store_item['answer_cluster'] = question['select_design_concept']['cluster']
            store_item['answer_pos_tag'] = question['select_design_concept']['pos_tag']
            other_options = []
            for i in range(3):
                key = 'Option'+str(i+1)
                if good_question[key] != good_question['Correct Answer']:
                    other_options.append(good_question[key])
            store_item['other_option_1'] = other_options[0]
            store_item['other_option_2'] = other_options[1]

            explanation = question['explanation']
            text = question['question']
            right_answer = question['right_answer']
            len_blank = len(right_answer)
            replaced_term = '_'*len_blank
            critique = text.replace(replaced_term, good_question['Correct Answer'])
            find_critique = critique[:-1]
            store_item['question'] = text.replace(replaced_term, '____')


            critique_index_start = explanation.find(critique)
            critique_index_end = critique_index_start + len(critique)
    #         print(critique_index_start)
    #         print(critique_index_end)
    # #         print(explanation)
    #         print(explanation[critique_index_start:critique_index_end])

            hint = '\"<b>' + store_item['question'] + '</b>\"'
            explanation = '\"<b>' + store_item['question'] + '</b>\"'

            comment = {}
            comment_raw = ''
            for comment_temp in item['comments']:
                comment_summary = comment_temp['comment_summary']
    #             print('*******')
    #             print(comment_summary)
    #             print(critique)
                temp = [index for (index , item) in enumerate(sent_tokenize(comment_summary)) if (find_critique.find(item.split('.')[0]) >= 0 or find_critique.find(item.split('?')[0]) >= 0 or find_critique.find(item.split('!')[0]) >= 0 or find_critique.find(item.split(';')[0]) >= 0)]
                if len(temp) > 0:
                    comment_raw = comment_temp['comment_raw']
                    comment = comment_temp
                    break

            # 删除bot留下的comment
            if 'Welcome to UI Design.' in comment_raw:
                continue


            # 定位critique在comment_summary中的位置。
            raw_sentences =  sent_tokenize(comment_raw)
            summary_sentences =  sent_tokenize(comment_summary)
            critique_index = [index for (index , item) in enumerate(summary_sentences) if 
                              (critique.find(item.split('.')[0]) >= 0 or critique.find(item.split('?')[0]) >= 0 
                                or critique.find(item.split('!')[0]) >= 0 or critique.find(item.split(';')[0]) >= 0
                                or item.find(find_critique) >= 0
                               )]
    #         print(critique_index)
    #         print(summary_sentences)
            critique_in_raw = [index for (index , item) in enumerate(raw_sentences) if 
                               (critique.find(item.split('.')[0]) >= 0 or critique.find(item.split('?')[0]) >= 0 
                                or critique.find(item.split('!')[0]) >= 0 or critique.find(item.split(';')[0]) >= 0
                                or item.find(find_critique) >= 0
                               )]
    #         print(critique_in_raw)
    #         print(raw_sentences)
    #         print('#'*30)
    #         print('comment_summary: {}'.format(comment_summary))
    #         print('comment_raw: {}'.format(comment_raw))
    #         print('critique: {}'.format(critique))
    # #         print(comment)
    #         print(critique_index)
    #         print(critique_in_raw)
            if comment == {} or len(critique_index) == 0 or len(critique_in_raw) == 0:
                continue

            critique_in_comment = []
            raw_critique_in_comment = []
            suggestion_in_comment = [] #
            raw_suggestion_in_comment = []
            rationale_in_comment = []
            raw_rationale_in_comment = []
            for i in range(len(comment['critique'])):
                sentence = comment['critique'][i]['sentence']
                raw_critique_in_comment.append(sentence)
                sentence = '<font color="#000000">' + sentence + '</font>'
                for j in range(len(comment['critique'][i]['ui_elements'])):
                    name = comment['critique'][i]['ui_elements'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
                for j in range(len(comment['critique'][i]['concepts'])):
                    name = comment['critique'][i]['concepts'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
                critique_in_comment.append(sentence)
            for i in range(len(comment['suggestion'])):
                sentence = comment['suggestion'][i]['sentence']
                raw_suggestion_in_comment.append(sentence)
                sentence = '<font color="#2F3756">' + sentence + '</font>'
                for j in range(len(comment['suggestion'][i]['ui_elements'])):
                    name = comment['suggestion'][i]['ui_elements'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
                for j in range(len(comment['suggestion'][i]['concepts'])):
                    name = comment['suggestion'][i]['concepts'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
                suggestion_in_comment.append(sentence)
            for i in range(len(comment['rationale'])):
                sentence = comment['rationale'][i]['sentence']
                raw_rationale_in_comment.append(sentence)
                sentence = '<font color="#CC0000">' + sentence + '</font>'
                for j in range(len(comment['rationale'][i]['ui_elements'])):
                    name = comment['rationale'][i]['ui_elements'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
                for j in range(len(comment['rationale'][i]['concepts'])):
                    name = comment['rationale'][i]['concepts'][j]['name']
                    if name != 'color' and name != 'font':
                        sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
                rationale_in_comment.append(sentence)

            if len(critique_index) > 0:
                locate_index = critique_in_raw[0]
                if locate_index == 0: 
                    if len(raw_sentences) > 1:
                        hint_add = raw_sentences[critique_in_raw[0]+1]
    #                     print('*********')
    #                     print(hint_add)
    #                     print(raw_sentences)
    #                     print(raw_critique_in_comment)
    #                     print(raw_suggestion_in_comment)
    #                     print(raw_rationale_in_comment)
                        temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
                        temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
                        temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
                        if len(temp1) > 0:
                            hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
                        elif len(temp2) > 0:
                            hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
                        elif len(temp3) > 0:
                            hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]
                        hint = hint + hint_add
                else:
                    hint_add = raw_sentences[critique_in_raw[0]-1]
                    temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
                    temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
                    temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
                    if len(temp1) > 0:
                        hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
                    elif len(temp2) > 0:
                        hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
                    elif len(temp3) > 0:
                        hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]
                    hint = hint_add + hint
    #                 print(len(raw_sentences))
    #                 print(locate_index)
    #                 print(critique_in_raw[0])
                    if len(raw_sentences) - locate_index > 1:
                        hint_add = raw_sentences[locate_index+1]
                        temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
                        temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
                        temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
                        if len(temp1) > 0:
    #                         print(critique_in_comment)
    #                         print(raw_critique_in_comment)
                            hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
                        elif len(temp2) > 0:
                            hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
                        elif len(temp3) > 0:
                            hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]

                        hint = hint + hint_add

    #         print(raw_sentences)
            print('hint: {}'.format(hint))

            explanation = ''
    #         print(comment_summary)
            for sen in summary_sentences:
                temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if item.find(sen) >= 0]
                temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if item.find(sen) >= 0]
                temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if item.find(sen) >= 0]
                if len(temp1) > 0:
                    hint_add = ' <b>(critique)</b> ' + critique_in_comment[temp1[0]]
                    if critique.find(sen) >= 0:
                        hint_add = '\"<b>' + hint_add + '</b>\"'
                    explanation += hint_add
                elif len(temp2) > 0:
                    hint_add = ' <b>(suggestion)</b> ' + suggestion_in_comment[temp2[0]]
                    explanation += hint_add
                elif len(temp3) > 0:
                    hint_add = ' <b>(rationale)</b> ' + rationale_in_comment[temp3[0]]
                    explanation += hint_add
            print('*' * 25)
            print(summary_sentences)
            print(raw_critique_in_comment)
            print(raw_suggestion_in_comment)
            print(raw_rationale_in_comment)
            print(comment_summary)
            print(comment_raw)
            print(explanation)
            print(question['explanation'])
            explanation = explanation + '\nFor more about this UI example, please check this <a href=\"' + item['post_url'] + '\" target="_blank">link</a>.'
            store_item['explanation'] = explanation
            store_item['hint'] = hint
            store_items.append(store_item)

            # es.index(index='question_pool', body=store_item)
            # print(doc)
    #         count += 1
        if count % 100 == 0:
            print('Insert {}/{} palettes: take {}s'.format(count, items_length, round(time.time() - start_time, 2)))
    # if count == 401:
        #     break

Insert 0/190 palettes: take 9163.2s
hint: It’s hard to appreciate the left screenshot because it bleeds into the faux background but you are focusing too much on the art and not enough on the purpose of the screen."<b>The fuzz test (make it ____; or zoom out and cross your eyes; is it still clear what to do?) doesn’t make it clear what to do in that page.</b>"doesn’t make it clear what to do in that page.
*************************
['I like the illustrations and colour scheme, and UX frills.', 'But I think you are focusing on the wrong things.', 'No one cares about the pixels if they can’t get the job done.', 'if I were your manager I would tell you to start again with a wireframe or paper mock up to get the functional flow; only then reintroduce the illustration and branding.', 'That’s because I worked in publishing and we almost required everyone to tattoo “Kill your darlings” on their foreheads.', 'Login button should be close to dead centre.', 'The fuzz test (make it blurry; or zoom

hint:  (suggestion) <font color="#2F3756">I would get rid of the <font color="#660099">shadow</font> and use an <font color="#660000">outline</font> and increasing the <font color="#660099">size</font> and the <font color="#660099">padding</font> of those <font color="#660000">elements</font> to make it easier to tap select.</font>"<b>Also, I don't get why the search icon is ____.</b>" (critique) <font color="#000000">On the third screen, the <font color="#660000">category names</font> are barely <font color="#660099">leg</font>ible</font>
*************************
['Regarding the UI, I found some things that I would change to improve it even further, for example, the shadow you used on the searchbar and the also in the second screen make the design looks a bit inconsistent.', 'I would get rid of the shadow and use an outline and increasing the size and the padding of those elements to make it easier to tap select.', "Also, I don't get why the search icon is red.", 'On the third screen

hint:  (critique) <font color="#000000">This is a bigger issue than just “<font color="#660099">spacing</font> isn’t perfect”.</font>"<b>Your ____ grouping and spacing is not correct.</b>" (suggestion) <font color="#2F3756">Your <font color="#660000">titles</font> should be closer to the <font color="#660000">items</font> they represent than the items they don’t.</font>
*************************
['This is a bigger issue than just “spacing isn’t perfect”.', 'Your vertical grouping and spacing is not correct.', 'Your titles should be closer to the items they represent than the items they don’t.', 'The arrows on the right should be larger and closer.', 'There are contrast issues with the arrows and the gradient.', 'And contrast issues with the color progress bar and the two images the progress bars are in a new location at']
['This is a bigger issue than just “spacing isn’t perfect”.', 'Your vertical grouping and spacing is not correct.', 'There are contrast issues with the arrows and the

hint: Love it."<b>But to others points yes ____ is kind and using it for hours is a consideration.</b>"That’s why so many enterprise softwares look a bit drab but they are also very functionally useful and the design supports call outs with tiny touches of color to draw the eye and make things easy to distinguish vs color everywhere.
*************************
['For palette a fantastic tool is colors.co it generates mathematically complementary color families.', 'But to others points yes contrast is kind and using it for hours is a consideration.', 'The design supports call outs with tiny touches of color to draw the eye and make things easy to distinguish vs color everywhere.', 'If folks will be using long form body copy in anyway, consider color carefully.', 'Medium is not white and off black by mistake.', 'Font size also matters.']
['But to others points yes contrast is kind and using it for hours is a consideration.', 'Medium is not white and off black by mistake. Font size also mat

So there are a lot of issues and it being “outdated” definitely isn’t main issue. There isn’t consistency in you design. The most obvious issues are the typography, the composition, and the spacing. It doesn’t look like you used a grid or layout at all.Typography: First off the text isn’t well-balanced. The typography for the headings does not work well together. Why are you changing the color for the Serif? Be consistent. Either use one color for all of the Serifs and a different color for the San-Serif or use one font for the headings with two different colors. Not both. The leading/line-height on essentially everything is off. For example the first header, you clearly see that the last line has a greater line-height. And The tagline doesn’t have enough of a line-height. It’s too tight. All of the leading for the headings should look similar, perceived similarly, regardless of font size. Like compare the top heading with “Try Test With our 30 Day Trail” and “The solution that…”. They

In [94]:
# store_items = []
# for item in items:
#     for question in item['questions']:
#         # if 'other_options_wiki' in question.keys():
#         store_item = copy.deepcopy(item)
#         # print(question)
#         store_item['questions'] = ''
#         store_item['question_id:'] = question['question_id:']
#         store_item['raw_critique'] = question['raw_critique']
#         store_item['select_design_concept'] = question['select_design_concept']
#         store_item['mention_ui_elements'] = question['mention_ui_elements']
#         store_item['question'] = question['question']
#         store_item['right_answer'] = question['right_answer']
#         store_item['explanation'] = question['explanation']
#         # store_item['other_options'] = question['other_options']
#         # store_item['other_options_wiki'] = question['other_options_wiki']
        
#         store_item['ui_elements'] = ''
#         for ui_element in question['mention_ui_elements']:
#             store_item['ui_elements'] += ' ' + ui_element['name']

#         # new structure 20220820
#         store_item['answer_cluster'] = question['select_design_concept']['cluster']
#         store_item['answer_pos_tag'] = question['select_design_concept']['pos_tag']
#         store_item['other_option_1'] = question['other_options'][0]
#         store_item['other_option_2'] = question['other_options'][1]

#         explanation = question['explanation']
#         text = question['question']
#         right_answer = question['right_answer']
#         len_blank = len(right_answer)
#         replaced_term = '_'*len_blank
#         critique = text.replace(replaced_term, right_answer)
#         find_critique = critique[:-1]
#         store_item['question'] = text.replace(replaced_term, '____')
        
        
#         critique_index_start = explanation.find(critique)
#         critique_index_end = critique_index_start + len(critique)
# #         print(critique_index_start)
# #         print(critique_index_end)
# # #         print(explanation)
# #         print(explanation[critique_index_start:critique_index_end])

#         hint = '\"<b>' + store_item['question'] + '</b>\"'
#         explanation = '\"<b>' + store_item['question'] + '</b>\"'
    
#         comment = {}
#         comment_raw = ''
#         for comment_temp in item['comments']:
#             comment_summary = comment_temp['comment_summary']
# #             print('*******')
# #             print(comment_summary)
# #             print(critique)
#             temp = [index for (index , item) in enumerate(sent_tokenize(comment_summary)) if (find_critique.find(item.split('.')[0]) >= 0 or find_critique.find(item.split('?')[0]) >= 0 or find_critique.find(item.split('!')[0]) >= 0 or find_critique.find(item.split(';')[0]) >= 0)]
#             if len(temp) > 0:
#                 comment_raw = comment_temp['comment_raw']
#                 comment = comment_temp
#                 break
        
#         # 删除bot留下的comment
#         if 'Welcome to UI Design.' in comment_raw:
#             continue

            
#         # 定位critique在comment_summary中的位置。
#         raw_sentences =  sent_tokenize(comment_raw)
#         summary_sentences =  sent_tokenize(comment_summary)
#         critique_index = [index for (index , item) in enumerate(summary_sentences) if 
#                           (critique.find(item.split('.')[0]) >= 0 or critique.find(item.split('?')[0]) >= 0 
#                             or critique.find(item.split('!')[0]) >= 0 or critique.find(item.split(';')[0]) >= 0
#                             or item.find(find_critique) >= 0
#                            )]
# #         print(critique_index)
# #         print(summary_sentences)
#         critique_in_raw = [index for (index , item) in enumerate(raw_sentences) if 
#                            (critique.find(item.split('.')[0]) >= 0 or critique.find(item.split('?')[0]) >= 0 
#                             or critique.find(item.split('!')[0]) >= 0 or critique.find(item.split(';')[0]) >= 0
#                             or item.find(find_critique) >= 0
#                            )]
# #         print(critique_in_raw)
# #         print(raw_sentences)
# #         print('#'*30)
# #         print('comment_summary: {}'.format(comment_summary))
# #         print('comment_raw: {}'.format(comment_raw))
# #         print('critique: {}'.format(critique))
# # #         print(comment)
# #         print(critique_index)
# #         print(critique_in_raw)
#         if comment == {} or len(critique_index) == 0 or len(critique_in_raw) == 0:
#             continue
        
#         critique_in_comment = []
#         raw_critique_in_comment = []
#         suggestion_in_comment = [] #
#         raw_suggestion_in_comment = []
#         rationale_in_comment = []
#         raw_rationale_in_comment = []
#         for i in range(len(comment['critique'])):
#             sentence = comment['critique'][i]['sentence']
#             raw_critique_in_comment.append(sentence)
#             sentence = '<font color="#000000">' + sentence + '</font>'
#             for j in range(len(comment['critique'][i]['ui_elements'])):
#                 name = comment['critique'][i]['ui_elements'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
#             for j in range(len(comment['critique'][i]['concepts'])):
#                 name = comment['critique'][i]['concepts'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
#             critique_in_comment.append(sentence)
#         for i in range(len(comment['suggestion'])):
#             sentence = comment['suggestion'][i]['sentence']
#             raw_suggestion_in_comment.append(sentence)
#             sentence = '<font color="#2F3756">' + sentence + '</font>'
#             for j in range(len(comment['suggestion'][i]['ui_elements'])):
#                 name = comment['suggestion'][i]['ui_elements'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
#             for j in range(len(comment['suggestion'][i]['concepts'])):
#                 name = comment['suggestion'][i]['concepts'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
#             suggestion_in_comment.append(sentence)
#         for i in range(len(comment['rationale'])):
#             sentence = comment['rationale'][i]['sentence']
#             raw_rationale_in_comment.append(sentence)
#             sentence = '<font color="#CC0000">' + sentence + '</font>'
#             for j in range(len(comment['rationale'][i]['ui_elements'])):
#                 name = comment['rationale'][i]['ui_elements'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660000">' + name + '</font>', 1)
#             for j in range(len(comment['rationale'][i]['concepts'])):
#                 name = comment['rationale'][i]['concepts'][j]['name']
#                 if name != 'color':
#                     sentence = sentence.replace(name, '<font color="#660099">' + name + '</font>', 1)
#             rationale_in_comment.append(sentence)
        
#         if len(critique_index) > 0:
#             locate_index = critique_in_raw[0]
#             if locate_index == 0: 
#                 if len(raw_sentences) > 1:
#                     hint_add = raw_sentences[critique_in_raw[0]+1]
# #                     print('*********')
# #                     print(hint_add)
# #                     print(raw_sentences)
# #                     print(raw_critique_in_comment)
# #                     print(raw_suggestion_in_comment)
# #                     print(raw_rationale_in_comment)
#                     temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
#                     temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
#                     temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
#                     if len(temp1) > 0:
#                         hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
#                     elif len(temp2) > 0:
#                         hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
#                     elif len(temp3) > 0:
#                         hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]
#                     hint = hint + hint_add
#             else:
#                 hint_add = raw_sentences[critique_in_raw[0]-1]
#                 temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
#                 temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
#                 temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
#                 if len(temp1) > 0:
#                     hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
#                 elif len(temp2) > 0:
#                     hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
#                 elif len(temp3) > 0:
#                     hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]
#                 hint = hint_add + hint
# #                 print(len(raw_sentences))
# #                 print(locate_index)
# #                 print(critique_in_raw[0])
#                 if len(raw_sentences) - locate_index > 1:
#                     hint_add = raw_sentences[locate_index+1]
#                     temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if hint_add.find(item) >= 0]
#                     temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if hint_add.find(item) >= 0]
#                     temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if hint_add.find(item) >= 0]
#                     if len(temp1) > 0:
# #                         print(critique_in_comment)
# #                         print(raw_critique_in_comment)
#                         hint_add = ' (critique) ' + critique_in_comment[temp1[0]]
#                     elif len(temp2) > 0:
#                         hint_add = ' (suggestion) ' + suggestion_in_comment[temp2[0]]
#                     elif len(temp3) > 0:
#                         hint_add = ' (rationale) ' + rationale_in_comment[temp3[0]]
                    
#                     hint = hint + hint_add
                
# #         print(raw_sentences)
#         print('hint: {}'.format(hint))
       
#         explanation = ''
# #         print(comment_summary)
#         for sen in summary_sentences:
#             temp1 = [index for (index , item) in enumerate(raw_critique_in_comment) if item.find(sen) >= 0]
#             temp2 = [index for (index , item) in enumerate(raw_suggestion_in_comment) if item.find(sen) >= 0]
#             temp3 = [index for (index , item) in enumerate(raw_rationale_in_comment) if item.find(sen) >= 0]
#             if len(temp1) > 0:
#                 hint_add = ' <b>(critique)</b> ' + critique_in_comment[temp1[0]]
#                 if critique.find(sen) >= 0:
#                     hint_add = '\"<b>' + hint_add + '</b>\"'
#                 explanation += hint_add
#             elif len(temp2) > 0:
#                 hint_add = ' <b>(suggestion)</b> ' + suggestion_in_comment[temp2[0]]
#                 explanation += hint_add
#             elif len(temp3) > 0:
#                 hint_add = ' <b>(rationale)</b> ' + rationale_in_comment[temp3[0]]
#                 explanation += hint_add
#         print('*' * 25)
#         print(summary_sentences)
#         print(raw_critique_in_comment)
#         print(raw_suggestion_in_comment)
#         print(raw_rationale_in_comment)
#         print(comment_summary)
#         print(comment_raw)
#         print(explanation)
#         print(question['explanation'])
#         explanation = explanation + '\nFor more about this UI example, please check this <a href=\"' + item['post_url'] + '\" target="_blank">link</a>.'
#         store_item['explanation'] = explanation
#         store_item['hint'] = hint
#         store_items.append(store_item)
        
#         # es.index(index='question_pool', body=store_item)
#         # print(doc)
#         count += 1
#         if count % 100 == 0:
#             print('Insert {}/{} palettes: take {}s'.format(count, items_length, round(time.time() - start_time, 2)))
#     # if count == 401:
#         #     break

In [95]:
print(count)

492


In [96]:
with open('./question_pool_0825.json', 'w') as f:
    json_string = json.dumps(store_items)
    f.write(json_string)

In [67]:
# users = [{
#     'user_id': 0, #记录是哪个user
#     'questions': [{
#         'question_id': 0,
#         'post_id': '0',
#         'user_answer': '0',
#         'grade': 0
#     }], #所有回答过的question information
#     'correct': 0, # 记录回答正确的条数
#     'total': 0, # 记录总共回答过的条数
    
# }]
# with open('./user_record.json', 'w') as f:
#     json_string = json.dumps(users)
#     f.write(json_string)

In [83]:
# import pandas as pd

# # convert into dataframe
# df = pd.read_excel("./quiz1_process.xlsx")
# dataset = []
# # convert into dictionary
# columns = df.to_dict('records')
# print(columns[0])

{'Question': 'The fuzz test (make it ______; or zoom out and cross your eyes; is it still clear what to do?) doesn’t make it clear what to do in that page.', 'Option1': 'blurry', 'Option2': 'colorful', 'Option3': 'contrasting', 'Correct Answer': 'blurry'}


In [2]:
import sys
import json
sys.path.insert(1, './question_base')
from query_question_pool import *
query_results = search_question('')
# print(query_results[0])
all_ui_elements = []
all_visual_elements = []
for question in query_results:
    if question['right_answer'] not in all_visual_elements:
        all_visual_elements.append(question['right_answer'])
    if len(question['mention_ui_elements']) > 0:
        for element in question['mention_ui_elements']:
            if element['name'] not in all_ui_elements:
                all_ui_elements.append(element['name'])
print(len(all_ui_elements)) #297
print(len(all_visual_elements)) # 176
with open('./all_ui_elements.json', 'w') as f:
    json_string = json.dumps(all_ui_elements)
    print(json_string)
    f.write(json_string)
with open('./all_visual_elements.json', 'w') as f:
    json_string = json.dumps(all_visual_elements)
    print(json_string)
    f.write(json_string)

337
296
166
["profile icon", "\"Koordinatore\"", "button", "list-item", "labels", "texts", "fields", "white text", "text", "left of your screen", "card", "options", "inputs", "Transactions heading", "left hand side of the view port", "promo material", "app stores", "layer", "article", "rectangle", "dashboard", "'", "border", "right-hand-side", "rounded corner", "border radius", "charts", "follow button", "nav bar", "\"subscriptions\" items", "hero post", "image", "global menu", "category tab", "search icon", "entries", "note", "left", "date", "left margin of the calendar numbers", "left margin of the days of the week", "word", "\u201cCalendar\u201d", "numbers", "Right side", "balance", "menu items", "date and time", "The right side", "logo", "primary content background", "Buttons", "components", "background", "icons", "pages", "logic", "buttons", "icon", "parcel", "vertical 8", "system", "sides", "ui elements", "top right of", "sign in button", "cta", "toggle", "fruit", "elements", "to

In [114]:
# import sys
# sys.path.insert(1, './question_base')
# from query_question_pool import *
# query_results = search_question('')
# # print(query_results[0])
# format_design = []
# layout_design = []
# visual_design = []
# color_design = []
# for question in query_results:
#     select_design_concept = question['select_design_concept']
#     if select_design_concept['cluster'] == 'format_design':
#         if question['right_answer']  not in format_design:
#             format_design.append(question['right_answer'])
#     if select_design_concept['cluster'] == 'layout_design':
#         if question['right_answer']  not in layout_design:
#             layout_design.append(question['right_answer'] )
#     if select_design_concept['cluster'] == 'visual_design':
#         if question['right_answer']  not in visual_design:
#             visual_design.append(question['right_answer'] )
#     if select_design_concept['cluster'] == 'color_design':
#         if question['right_answer']  not in color_design:
#             color_design.append(question['right_answer'] )
# print(len(format_design)) # 135  如果算不重复的，则是  84
# print(len(layout_design)) # 23   13
# print(len(visual_design)) # 93   47
# print(len(color_design)) # 89   39
# print(format_design)
# print(layout_design)
# print(visual_design)
# print(color_design)
# # with open('./all_ui_elements.json', 'w') as f:
# #     json_string = json.dumps(all_ui_elements)
# #     print(json_string)
# #     f.write(json_string)
# # with open('./all_visual_elements.json', 'w') as f:
# #     json_string = json.dumps(all_visual_elements)
# #     print(json_string)
# #     f.write(json_string)

340
84
13
47
39
['white', 'alignment', 'whitespace', 'visual', 'aligned', 'aligns', 'margin', 'size', 'space', 'width', 'signup', 'placement', 'font', 'cohesion', 'align', 'spacing', 'weighting', 'lay', 'paddings', 'different', 'bevel', 'duration', 'horizontal', 'centered', 'scale', 'rectangles,', 'padding', 'margins', 'Spacing', 'over', 'floating', 'Skeuomorphism', 'sizes', 'misalignment', 'letter', 'length', 'straight', 'alignments', 'Alignment', 'text', 'cleanliness', 'rounded', 'distance', 'sizing', 'mature', 'column', 'dimensions', 'Text', 'typographic', 'Placement', 'less', 'vertical', 'proximity', 'center', 'flat', 'general', 'form', 'legible', 'floating”', 'radius,', 'frame', 'graph', 'pastels,', 'room', '-radius)', 'White', 'norm', 'Cohesion', 'clutter', 'shape', 'Button', 'height', 'square', 'typo', 'Centered', 'Center', 'consistency', 'line-height', 'paper', 'position', 'usability,', 'cropping', 'clean,', 'street']
['readability', 'design', 'accessibility', 'usability', 'leg

In [119]:
# from datetime import datetime
# from datetime import timedelta
# from datetime import timezone
# SHA_TZ = timezone(
#     timedelta(hours=8),
#     name='Asia/Shanghai',
# )
# utc_now = datetime.utcnow().replace(tzinfo=timezone.utc)
# beijing_now = utc_now.astimezone(SHA_TZ)
# print(beijing_now, beijing_now.tzname())
# print(str(beijing_now) + str(beijing_now.tzname()))
# # print(beijing_now.date(), beijing_now.tzname())

2022-08-26 17:07:46.418778+08:00 Asia/Shanghai
2022-08-26 17:07:46.418778+08:00Asia/Shanghai


In [3]:
import sys
sys.path.insert(1, './question_base')
from query_question_pool import *
query_results = search_question('')
# for question in query_results:
#     if question['right_answer'] == 'Text':
#         break
# print(question['question'])
# print(question['_id'])
# from elasticsearch import Elasticsearch
# import random

print(len(query_results))
post_ids = []
# print(query_results[0])
for item in query_results:
    if item['post_id'] not in post_ids:
        post_ids.append(item['post_id'])
print(len(post_ids))
# es = Elasticsearch('http://localhost:9200')
# result = es.search(index='question_pool_all', body=dsl)
# result = es.delete(index='question_pool_all', size=search_size, body=dsl)

337
337
152
